In [1]:
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [5]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt="write strictly 'Yes' or 'No' to the following question: Will bitcoin exceed $80k today?\nAnswer: ",
    max_tokens=5,
    temperature=0,
    stream=False,
    logprobs=20,
)

In [6]:
from openai import OpenAI

def get_probability_from_openai(
        prompt,
        model='gpt-3.5-turbo-instruct',
        temperature=0,
        max_tokens=5,
        logprobs=20,
):
    
    client = OpenAI()
    response = client.completions.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        stream=False,
        logprobs=logprobs,
    )
    return response

In [7]:
response = get_probability_from_openai(
    prompt="write strictly 'Yes' or 'No' to the following question: Will bitcoin exceed $80k today?\nAnswer: ",
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    max_tokens=5,
    logprobs=20,
)

In [8]:
from pprint import pprint
pprint(response)

Completion(id='cmpl-BOu0rxIsHrB3A7Xzh7ga1vuWlLVmc', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=Logprobs(text_offset=[96], token_logprobs=[-0.055891387], tokens=['No'], top_logprobs=[{'No': -0.055891387, ' No': -3.5202334, '\n': -3.7771533, '\n\n': -7.863917, 'I': -8.268557, 'NO': -9.060968, ' \n': -9.269325, 'N': -9.302616, 'Not': -9.43722, 'Yes': -9.446232, 'Unknown': -9.52006, 'Sorry': -9.71632, "'": -9.733854, 'It': -9.91313, 'Today': -10.207884, 'no': -10.24527, 'There': -10.289873, 'Cannot': -10.301643, " '": -10.302044, 'Unfortunately': -10.307383}]), text='No')], created=1745274885, model='gpt-3.5-turbo-instruct:20230824-v2', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=1, prompt_tokens=25, total_tokens=26))


In [9]:
print(response.choices[0].logprobs.top_logprobs)

[{'No': -0.055891387, ' No': -3.5202334, '\n': -3.7771533, '\n\n': -7.863917, 'I': -8.268557, 'NO': -9.060968, ' \n': -9.269325, 'N': -9.302616, 'Not': -9.43722, 'Yes': -9.446232, 'Unknown': -9.52006, 'Sorry': -9.71632, "'": -9.733854, 'It': -9.91313, 'Today': -10.207884, 'no': -10.24527, 'There': -10.289873, 'Cannot': -10.301643, " '": -10.302044, 'Unfortunately': -10.307383}]


In [10]:
def extract_yes_no_logits_and_softmax(response):
    import numpy as np

    yes_logits = []
    no_logits = []
    yes_tokens = []
    no_tokens = []

    top_logprobs = response.choices[0].logprobs.top_logprobs[0]
    for token, logit in top_logprobs.items():
        processed_token = token.lower().strip()
        if processed_token == 'yes':
            yes_logits.append(logit)
            yes_tokens.append(token)
        elif processed_token == 'no':
            no_logits.append(logit)
            no_tokens.append(token)

    relevant_logits = np.array(yes_logits + no_logits)

    if relevant_logits.size == 0:
        return Exception("No relevant logits found for 'yes'S or 'no' tokens.")

    exp_logits = np.exp(relevant_logits - np.max(relevant_logits))
    softmax_probs = exp_logits / exp_logits.sum()

    yes_prob_sum = float(np.sum(softmax_probs[:len(yes_logits)])) if yes_logits else 0.0
    no_prob_sum = float(np.sum(softmax_probs[len(yes_logits):])) if no_logits else 0.0

    return {
        "yes_tokens_found": yes_tokens,
        "no_tokens_found": no_tokens,
        "yes_prob_sum": yes_prob_sum,
        "no_prob_sum": no_prob_sum,
        "softmax_probs_yes_no": softmax_probs.tolist()
    }

In [11]:
extract_yes_no_logits_and_softmax(response)

{'yes_tokens_found': ['Yes'],
 'no_tokens_found': ['No', ' No', 'NO', 'no'],
 'yes_prob_sum': 8.097330785986965e-05,
 'no_prob_sum': 0.9999190266921401,
 'softmax_probs_yes_no': [8.097330785986965e-05,
  0.9694267348473401,
  0.03033684222143434,
  0.00011903095290926953,
  3.641867045648171e-05]}